# Django Tutorial
[See Django Documentation Contents]( https://docs.djangoproject.com/ko/3.0/)

### Virtual env에서 Django package 설치하기
venv는 workspace 내에 만들거나 workspace 밖에 만들 수 있다. 밖에 만들면 다른 project와 이 가상환경을 공유하려면 그렇게 하는 편이 좋다. 어느 쪽이든 version관리 대상이 아니다. 왜냐 하면, 다른 컴퓨터로 사본을 이전할 이유가 없다. 
- 환경을 이전하려면, 'requirements.txt' 파일만 이전하면 충분하다.) 

여기서는 workspace 내에 가상환경 이름을 venv로 만들기로 한다. 
venv 환경을 생성하여 activate한다.
- 참고: .venv venv env .env 등은 통상적으로 .gitignore에 삽입하는 편이 좋다. 

```
$ cd <workspace>
$ python -m venv venv
bash:$ source venv/Scripts/activate
---
powershell:> Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope CurrentUser
powershell:> venv\Scripts\Activate.ps1 
```

최신 Django package를 install한다.
```
(venv) $ pip install Django
```

특정 버전을 설치하려면 Django==3.0.6 처럼 버전을 기입.
Note: 이 가상환경에서 설치된 package들은 기존에 설치된 Python interpreter에 영향을 주지 않는다.

설치된 package들을 확인해 본다.
```
(venv) $ pip list
```

## Setup a database
PostgreSQL, MySQL, or Oracle 등 대형 DBMS:
- 별도 설치가 필요하다. 
- 이 DBMS는 application server와 별도의 machine에서 실행 가능하며 TCP socket 통신으로 query가 전달된다. 
- 물론, 사용권한이 있어야 한다.
- Python interface를 위한 DB API 설치 필요

>참고: Python MySQL https://www.w3schools.com/python/python_mysql_getstarted.asp

SQLite3:
- Django의 default DB이며 표준 Python에 포함되어 있다. 
- 다만, SQL을 사용하지 않고 DB를 access하려면, [SQLite Database Browser](https://sqlitebrowser.org/)를 설치하자.
- 참고: [SQLite로 가볍게 배우는 데이터베이스](https://wikidocs.net/book/1530) 

## Web Application Architectures
![web application architectures](https://reinvently.com/wp-content/uploads/2019/08/scheme.jpg)

## Django at a glance
https://docs.djangoproject.com/ko/3.0/intro/overview/

[Django 문서](https://docs.djangoproject.com/ko/3.0/)

Django Architecture (MVTC)
![Django Architecture](static/django_arch.jpg)

Django Layout
![Django Layout](static/layout.png)

## 첫 번째 장고 앱 작성하기, part 1 
https://docs.djangoproject.com/ko/3.0/intro/tutorial01/
###### Creating a project
```
$ django-admin startproject mysite
```

###### The development server
```
$ python manage.py runserver
```

###### Creating the Polls app
```
$ python manage.py startapp polls
```

###### Write your first view
- polls/views.py
- polls/urls.py
- mysite/urls.py

URLconfs
```
path(route, view, kwargs=None, name=None)
re_path(route, view, kwargs=None, name=None)
```

###### Run
Run server:
```
$ python manage.py runserver
```

Go to http://localhost:8000/polls/ in your browser


## 첫 번째 장고 앱 작성하기, part 2
https://docs.djangoproject.com/ko/3.0/intro/tutorial02/

설정 - `mysite/settings.py` 파일을 다음과 같이 수정한다. (comment 참조)
```Python
INSTALLED_APPS = [
    # Insert your app here
    # 'polls.apps.PollsConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    ...
    
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        # 'DIRS': [],
        'DIRS': [os.path.join(BASE_DIR, 'templates')],  # project's templates
        'APP_DIRS': True,               # lookup app's templates dir
        ...

# TIME_ZONE = 'UTC'
TIME_ZONE = 'Asia/Seoul'
```

###### Database setup
Django가 만들어 놓은 admin, auth 등 application들이 사용하는 DB schema가 미리 정의되어 있다. 이를 DB에 이동시킨다. 즉, db.sqlite3 DB에 table을 creation한다.
```
python manage.py migrate
```

###### Creating models
`polls/models`.py 파일를 작성한다. DB schema는 Python class로 대응된다. 

###### Activating models
`mysite/settings.py`에 `'polls.apps.PollsConfig'`을 추가하여 polls app을 등록한다.

Question, Choice table을 DB에 create시킬 `Migration` class를 생성한다.
```
$ python manage.py makemigrations polls
```

`polls/migrations/0001_initial.py`가 생성되었다. 여기에는 DB table을 create하기 위한 specification이 기술되어 있다.

>참고: 만든 migration을 가지고 DB에 table을 creationg하기 위해 어떠한 SQL statements 생성되는지 확인해 보자. 

>`$ python manage.py sqlmigrate polls 0001`


작성한 model(즉, Python class)에서 DB에 table을 생성하자. 변경된 migration들이 존재하면(즉, model에 변경이 있으면) DB에 이를 적용한다. (Table field에 변경이 있거나, 삭제 등)
```
$ python manage.py migrate
```

이제 class Question과 Choice는 DB에 polls_question, polls_choice라는 table명으로 생성되었다. 

*SQLite DB Browser*로 확인해 보자.

> `makemigrations [app_name]` 
- generates the sql command to create the table corresponding to each class you made in models.py file. 
- 모델이 변경됐을 때만, migrations/ 에 새로운 .py 파일이 생긴다.

> `migrate [app_name]`
- will create the table in database using the commands which have been generated by makemigrations.

> `sqlmigrate`:
- shows all the SQL commands which has been generated by makemigrations.

> `showmigrations`:
- show what migrations exist in a Django project

###### Playing with the API
Python shell로 interactive하게 DB에 insert, select할 수 있지만, 장고가 제공하는 database API를 이해하도록 하자. 실제로는 admin application에서 DB를 handling할 수 있으니 DB에 data 삽입은 미루기로 한다.

`polls/models.py`는 tutorial에 따라 수정한다.

###### Introducing the Django Admin
1. Creating an admin user
```
$ python manage.py createsuperuser
```
여러분의 user id와 e-mail, password를 등록한다.
2. Start the development server <br>
Development server를 run시키고 http://localhost:8000/admin 로 가서 loggin하자. 
3. Enter the admin site
4. Make the poll app modifiable in the admin
5. Explore the free admin functionality

###### 참고: Admin이 DB에 설문 입력하기
Admin이 browser에서 설문 데이터를 입력할 수 있게 다음 code로 대체해 보자. 

polls/admin.py:
```Python
from django.contrib import admin

from .models import Question, Choice

class ChoiceInline(admin.TabularInline):
    model = Choice
    extra = 3

class QuestionAdmin(admin.ModelAdmin):
    list_display = ('question_text', 'pub_date')
    list_filter = ['pub_date']
    inlines = [ChoiceInline]

admin.site.register(Question, QuestionAdmin)
```
Browser에서 몇 개의 설문을 입력하자.

## [Writing your first Django app, part 3](https://docs.djangoproject.com/en/2.2/intro/tutorial03/)
Overview

Writing more views

Write views that actually do something

Raising a 404 error

Use the template system

Removing hardcoded URLs in templates

Namespacing URL names

## [Writing your first Django app, part 4](https://docs.djangoproject.com/en/2.2/intro/tutorial04/)
Write a simple form

Use generic views: Less code is better

## [Writing your first Django app, part 5](https://docs.djangoproject.com/en/2.2/intro/tutorial05/)
Introducing automated testing

Basic testing strategies

Writing our first test

Test a view

When testing, more is better

Further testing

What’s next?

## [Writing your first Django app, part 6](https://docs.djangoproject.com/en/2.2/intro/tutorial06/)
Customize your app’s look and feel

Adding a background-image

## [Writing your first Django app, part 7](https://docs.djangoproject.com/en/2.2/intro/tutorial07/)

Customize the admin form

Adding related objects

Customize the admin change list

Customize the admin look and feel

Customize the admin index page

What’s next?

## Production and Deployment
### Use GitHub
Source code는 GitHub를 이용하면 이전이 가능하다. 단, database는 버전관리 대상이면 안된다. (.gitignore에 `*.sqlites3` 라인을 추가하자.)

### Database 이전
Database의 content는 machine과 OS에 따라 data가 다르게 표현될 수 있기 때문에 그대로 이식할 수 없다. 따라서 이식성이 있는 JSON으로 저장했다가 server에 다시 loading해야 한다.

Dump:
```
python ./manage.py dumpdata --exclude auth.permission --exclude contenttypes > db.json
```
> 인증과 contenttypes는 dump에서 제외해야한다.

Load:
```
python ./manage.py loaddata db.json
```

### Virtual Environment 재설정 
